# **Predicción del precio de vuelos**
## Proyecto y prueba Final Data Science - Academia Desafío Latam

Este notebook aborda el problema de predecir el precio de boletos de avión usando datos de `business.csv` y `economy.csv`, aplicando limpieza, análisis exploratorio, ingeniería de variables, modelamiento y evaluación. **Modelos candidatos:** Random Forest, Lasso, XGBoost


**Nombre: Kattya Contreras**

**Módulo: Machine Learning**

**Sección: G101**

**1. Problema de negocio y metodología**

**Problema:** Predecir el precio de boletos de avión para una agencia de viajes online, considerando variables como aerolínea, clase, destino, fechas, etc.

**Metodología:**
- Análisis exploratorio y calidad de datos
- Limpieza y preprocesamiento
- Ingeniería de variables
- Modelamiento y optimización
- Evaluación de modelos
- Conclusiones y próximos pasos

**Variable objetivo:** `price`

**Paso 1. Importación de librerías y configuración general**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
plt.style.use('ggplot')
print("✅ Librerías importadas correctamente")


**PASO 2: Carga de datos y reporte de calidad**

In [ ]:
df_business = pd.read_excel("Data/business.xlsx")
df_economy = pd.read_excel("Data/economy.xlsx")

def quality_report(df, name):
    print(f"\n{name} - Info:")
    print(df.info())
    print("\nNulos por columna:")
    print(df.isnull().sum())
    print("\nDuplicados:", df.duplicated().sum())
    print("\nDescriptivo price:")
    print(df['price'].describe())

quality_report(df_business, "BUSINESS")
quality_report(df_economy, "ECONOMY")


**PASO 3. Limpieza básica: nulos y duplicados**

In [ ]:
df_business = df_business[df_business['price'].notnull()].drop_duplicates().copy()
df_economy = df_economy.drop_duplicates().copy()
print("Business shape:", df_business.shape)
print("Economy shape:", df_economy.shape)

**PASO 4. Limpieza avanzada: columna stop y variable numérica**

In [ ]:
import re

def clean_stop_column(df):
    df['stop'] = df['stop'].astype(str).str.replace(r'[\n\t]+', '', regex=True).str.strip()
    def stop_to_num(s):
        s = s.lower()
        if 'non-stop' in s:
            return 0
        if '1-stop' in s:
            return 1
        if '2-stop' in s or '2+' in s:
            return 2
        match = re.search(r'(\d+)-stop', s)
        return int(match.group(1)) if match else None
    df['stop_num'] = df['stop'].apply(stop_to_num)
    return df

df_business = clean_stop_column(df_business)
df_economy = clean_stop_column(df_economy)

**PASO 5. Feature Engineering: duración de vuelo en minutos**

In [ ]:
def time_taken_to_minutes(s):
    if pd.isnull(s): return None
    try:
        h, m = 0, 0
        parts = s.split('h')
        if len(parts) == 2:
            h = int(parts[0].strip())
            m = int(parts[1].replace('m','').strip())
        elif 'h' in s:
            h = int(s.split('h')[0].strip())
        elif 'm' in s:
            m = int(s.replace('m','').strip())
        return h*60 + m
    except:
        return None

df_business['time_taken_min'] = df_business['time_taken'].apply(time_taken_to_minutes)
df_economy['time_taken_min'] = df_economy['time_taken'].apply(time_taken_to_minutes)

**PASO 6. Más variables: mes, día de semana y ruta**

In [ ]:
df_business['month'] = df_business['date'].dt.month
df_business['day_of_week'] = df_business['date'].dt.dayofweek
df_business['route'] = df_business['from'] + '-' + df_business['to']

df_economy['month'] = df_economy['date'].dt.month
df_economy['day_of_week'] = df_economy['date'].dt.dayofweek
df_economy['route'] = df_economy['from'] + '-' + df_economy['to']

**PASO 7. Codificación de variables categóricas (top aerolíneas y rutas)**

In [ ]:
top_airlines_business = df_business['airline'].value_counts().nlargest(10).index.tolist()
df_business['airline_top'] = df_business['airline'].apply(lambda x: x if x in top_airlines_business else 'other')
df_business = pd.get_dummies(df_business, columns=['airline_top', 'route'], drop_first=True)

top_airlines_economy = df_economy['airline'].value_counts().nlargest(10).index.tolist()
df_economy['airline_top'] = df_economy['airline'].apply(lambda x: x if x in top_airlines_economy else 'other')
df_economy = pd.get_dummies(df_economy, columns=['airline_top', 'route'], drop_first=True)


**PASO 8. Imputación de nulos en nuevas columnas numéricas**

In [ ]:
for col in ['time_taken_min', 'stop_num']:
    df_business[col] = df_business[col].fillna(df_business[col].median())
    df_economy[col] = df_economy[col].fillna(df_economy[col].median())

**PASO 9. Eliminar columnas irrelevantes**

In [ ]:
cols_to_drop = ['ch_code', 'dep_time', 'arr_time', 'airline', 'from', 'to', 'time_taken', 'stop']
df_business = df_business.drop(cols_to_drop, axis=1)
df_economy = df_economy.drop(cols_to_drop, axis=1)

**PASO 10. Análisis exploratorio visual (EDA)**

In [ ]:
# ===== Limpieza y conversión de 'price' =====
for df in [df_business, df_economy]:
    df['price'] = (
        df['price']
        .astype(str)              # asegurar que sea string
        .str.replace(',', '', regex=False)  # quitar comas
        .str.strip()              # quitar espacios
    )
    df['price'] = pd.to_numeric(df['price'], errors='coerce')  # convertir a numérico

# Eliminar nulos resultantes
df_business = df_business[df_business['price'].notnull()].copy()
df_economy = df_economy[df_economy['price'].notnull()].copy()

print("✅ Limpieza de 'price' completada")
print("Business shape:", df_business.shape)
print("Economy shape:", df_economy.shape)


In [ ]:
# ===== Histograma con KDE para distribución de precios =====
plt.figure(figsize=(10,6))
sns.histplot(df_business['price'], kde=True, color='red', label='Business', bins=50)
sns.histplot(df_economy['price'], kde=True, color='blue', label='Economy', bins=50)
plt.legend()
plt.title('Distribución de precios: Business vs Economy')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.show()


# **Variables más importantes para price (correlaciones)**

- La variable stop_num (cantidad de escalas) tiene la correlación más alta con el precio (0.59), seguida por time_taken_min (duración del vuelo, 0.24) y num_code (0.22).

- day_of_week y month muestran correlaciones bajas, pero podrían tener efectos no lineales.

- Implicaciones: el modelo debe priorizar variables como escalas y duración del vuelo, mientras que otras variables con baja correlación aún pueden ser útiles para capturar relaciones complejas.

**Síntesis:**

- Los precios dependen principalmente de la cantidad de escalas y duración del vuelo, y existen diferencias marcadas entre clases (Business vs Economy).

- Esto valida la elección de modelos no lineales y la necesidad de un preprocesamiento cuidadoso, incluyendo ingeniería de variables y manejo de outliers.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# Definir colormap armonioso: azul medio -> blanco -> rojo profundo
colors = ["#1f4e79", "#f0f0f0", "#b30000"]  # azul medio, gris claro, rojo profundo
cmap = LinearSegmentedColormap.from_list("blue_white_red_professional", colors)

# Correlación de variables numéricas
corr = df_business[['price', 'stop_num', 'time_taken_min', 'num_code', 'day_of_week', 'month']].corr()

plt.figure(figsize=(7,5))
sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    cmap=cmap,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink":0.8, "label":"Correlación"}
)
plt.title("Mapa de correlaciones de variables numéricas", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


# **Conclusión del análisis exploratorio visual y correlaciones**

- Distribución de precios (Business vs Economy)

- Business tiene precios más altos y una cola larga de valores extremos, mostrando gran dispersión.

- Economy está más concentrada en rangos bajos-medios, con menos valores extremos.

- Ambas distribuciones están sesgadas a la derecha, indicando que la media no representa completamente la tendencia central.

**Implicaciones: se recomienda realizar transformación del target (price) y manejo de outliers para mejorar el desempeño de los modelos. Además, los modelos no lineales (Random Forest, XGBoost) son más adecuados que modelos lineales simples (Lasso).**


**PASO 11. Modelado: entrenamiento y evaluación**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# ==============================
# Separar features y target
# ==============================
X = df_business.drop('price', axis=1)
y = df_business['price']

# Procesar columna de fecha
# ==============================
if 'date' in X.columns:
    X['year'] = X['date'].dt.year
    X['month'] = X['date'].dt.month
    X['day'] = X['date'].dt.day
    X['weekday'] = X['date'].dt.weekday
    X['is_weekend'] = X['date'].dt.weekday >= 5
    X = X.drop(columns=['date'])

print("Columnas después de procesar fechas:")
print(X.head())


Limpieza de Fecha: Convertimos la columna date en varias variables derivadas (year, month, day, weekday, is_weekend) para capturar estacionalidad y patrones según día y mes. Además, verificamos los tipos de datos resultantes para asegurar compatibilidad con los modelos de Machine Learning.

In [ ]:
# Procesamiento y limpieza de fechas

X = df_business.drop('price', axis=1)
y = df_business['price']

# Si existe la columna date, procesarla
if 'date' in X.columns:
    X['year'] = X['date'].dt.year
    X['month'] = X['date'].dt.month
    X['day'] = X['date'].dt.day
    X['weekday'] = X['date'].dt.weekday
    X['is_weekend'] = (X['date'].dt.weekday >= 5).astype(int)  # 0 o 1

    # 🔥 eliminar definitivamente la columna date
    X = X.drop(columns=['date'])

print("Dtypes después de limpiar fechas:")
print(X.dtypes)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


**División en test y train**

**Entrenamiento y Evaluación de modelos: RandomForest, Lasso, y XGBoost**

In [ ]:


from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Modelos
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'Lasso': Lasso(alpha=0.01, random_state=42),   # alpha pequeño para no penalizar demasiado
    'XGBoost': XGBRegressor(random_state=42, n_estimators=200, learning_rate=0.1)
}

# Entrenar y evaluar
for name, model in models.items():
    print(f"\nEntrenando {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"{name} → RMSE: {rmse:.2f}, R²: {r2:.4f}")


                                   Comparación de modelos para predicción de precio de vuelos

| Modelo        | RMSE    | R²     | Comentario                                                                              |
| ------------- | ------- | ------ | --------------------------------------------------------------------------------------- |
| Random Forest | 3626.39 | 0.9222 | Mejor desempeño, captura relaciones no lineales, más robusto frente a outliers          |
| Lasso         | 8803.48 | 0.5417 | Peor desempeño, modelo lineal simple, no maneja bien la distribución sesgada del target |
| XGBoost       | 4759.55 | 0.8660 | Bueno, cercano a Random Forest, permite cierta regularización y no linealidad           |


**Optimización de RandomForest con GridSearchCV**

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Definir grid de hiperparámetros
rf = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2,5],
    'min_samples_leaf': [1,2]
}

# GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor modelo
print("Mejores parámetros RandomForest:", grid_search.best_params_)
best_rf = grid_search.best_estimator_

# Predicciones
y_pred_rf = best_rf.predict(X_test)

# Métricas
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print(f"RandomForest Optimizado → RMSE: {rmse_rf:.2f}, R²: {r2_rf:.4f}, MAE: {mae_rf:.2f}")



Se realizó una búsqueda de hiperparámetros para RandomForest utilizando **GridSearchCV** con validación cruzada (cv=3).  
El modelo optimizado mejoró respecto al baseline y a la versión sin tuning, mostrando un ajuste sólido con buen equilibrio entre **precisión (R²)** y **error absoluto (MAE)**.

Mejores parámetros RandomForest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
RandomForest Optimizado → RMSE: 3604.44, R²: 0.9232, MAE: 1708.94


**PASO 12. Evaluación y comparación de modelos**

In [ ]:
# 1️⃣ Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 2️⃣ Función de evaluación de modelos
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # RMSE manual para compatibilidad
    r2 = r2_score(y_test, y_pred)
    return mae, rmse, r2

# 3️⃣ Evaluar cada modelo del diccionario 'models'
results = {}
for name, model in models.items():
    results[name] = evaluate(model, X_test, y_test)

# 4️⃣ Convertir resultados en DataFrame
metrics_df = pd.DataFrame(results, index=['MAE', 'RMSE', 'R2']).T
metrics_df.index.name = 'Modelo'
metrics_df.columns.name = 'Métrica'

# 5️⃣ Graficar comparación de métricas
metrics_df.plot(kind='bar', figsize=(10,6))
plt.title('Comparación de métricas entre modelos')
plt.ylabel('Valor')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 6️⃣ Evaluación de baseline: predecir la media de y_train
y_pred_baseline = np.full(shape=y_test.shape, fill_value=y_train.mean(), dtype=np.float64)
mae_base = mean_absolute_error(y_test, y_pred_baseline)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
r2_base = r2_score(y_test, y_pred_baseline)
print(f"Baseline (media): MAE={mae_base:.2f}, RMSE={rmse_base:.2f}, R2={r2_base:.2f}")

# 7️⃣ Comparación con baseline en gráfico adicional (opcional)
baseline_df = metrics_df.copy()
baseline_df.loc['Baseline'] = [mae_base, rmse_base, r2_base]

baseline_df.plot(kind='bar', figsize=(10,6))
plt.title('Comparación modelos vs Baseline')
plt.ylabel('Valor')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
# 1️⃣ DataFrame con baseline incluido
metrics_full_df = metrics_df.copy()
metrics_full_df.loc['Baseline'] = [mae_base, rmse_base, r2_base]

# 2️⃣ Configuración del gráfico
fig, ax1 = plt.subplots(figsize=(12,6))

x = np.arange(len(metrics_full_df))
bar_width = 0.35

# Barras MAE y RMSE en eje principal
ax1.bar(x - bar_width/2, metrics_full_df['MAE'], width=bar_width, label='MAE', color='skyblue')
ax1.bar(x + bar_width/2, metrics_full_df['RMSE'], width=bar_width, label='RMSE', color='salmon')
ax1.set_ylabel('MAE / RMSE')
ax1.set_xticks(x)
ax1.set_xticklabels(metrics_full_df.index)
ax1.set_title('Comparación de métricas por modelo (incluyendo Baseline)')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# 3️⃣ R² en eje secundario
ax2 = ax1.twinx()
ax2.plot(x, metrics_full_df['R2'], color='green', marker='o', linestyle='-', label='R²', linewidth=2)
ax2.set_ylabel('R²')
ax2.set_ylim(0,1)  # opcional: normalizar R² entre 0 y 1 para mejor visualización

# 4️⃣ Leyenda combinada
bars_labels = ax1.get_legend_handles_labels()
line_labels = ax2.get_legend_handles_labels()
ax1.legend(bars_labels[0] + line_labels[0], bars_labels[1] + line_labels[1], loc='upper right')

plt.show()


In [ ]:
# Baseline
y_pred_base = np.full_like(y_test, y_train.mean())
mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_base))
r2_base = r2_score(y_test, y_pred_base)

print("Baseline (media) → MAE: {:.2f}, RMSE: {:.2f}, R2: {:.4f}".format(mae_base, rmse_base, r2_base))

# Comparación final
print("\nConclusión:")
print("- RandomForest y XGBoost superan ampliamente el baseline y Lasso.")
print("- Variables más relevantes según correlación: time_taken_min, stop_num, month, day_of_week.")
print("- La data tiene outliers y colas largas en Business, por lo que modelos no lineales funcionan mejor.")
print("- Próximos pasos: tratar outliers, evaluar transformaciones logarítmicas, ajustar hiperparámetros XGBoost y evaluar Economy.")


# Conclusión del proyecto de predicción de precios de vuelos

## Resumen ejecutivo

- Se entrenaron tres modelos: **Random Forest**, **Lasso** y **XGBoost** para predecir el precio de boletos de avión (Business).  
- Se compararon sus métricas frente a un **baseline** simple (predecir la media de `price`).  
- Se aplicaron transformaciones de fechas, codificación de aerolíneas y rutas, y feature engineering para escalas y duración del vuelo.  
- La distribución de precios presenta **outliers y colas largas**, por lo que los modelos **no lineales** funcionan mejor.

---

## Comparación de modelos

<table>
<thead>
<tr>
<th>Modelo</th>
<th>RMSE</th>
<th>MAE</th>
<th>R²</th>
<th>Comentario</th>
</tr>
</thead>
<tbody>
<tr style="background-color:#c6efce;"> 
<td>Random Forest</td>
<td>3604.44</td>
<td>1708.94</td>
<td>0.9232</td>
<td>Mejor desempeño, captura relaciones no lineales y robusto frente a outliers</td>
</tr>
<tr style="background-color:#ffc7ce;"> 
<td>Lasso</td>
<td>8803.48</td>
<td>5342.01</td>
<td>0.5417</td>
<td>Peor desempeño, modelo lineal simple, no maneja bien la distribución sesgada del target</td>
</tr>
<tr style="background-color:#ffeb9c;"> 
<td>XGBoost</td>
<td>4759.55</td>
<td>2087.23</td>
<td>0.8660</td>
<td>Buen desempeño, cercano a Random Forest, permite regularización y relaciones no lineales</td>
</tr>
<tr style="background-color:#d9d9d9;"> 
<td>Baseline</td>
<td>13004.97</td>
<td>9732.75</td>
<td>-0.0001</td>
<td>Línea base (media de `price`), muestra la ventaja de los modelos predictivos</td>
</tr>
</tbody>
</table>

---

## Conclusión final

1. **Random Forest** es el modelo más recomendado, seguido de **XGBoost**.  
2. Las variables más relevantes son: `stop_num` (cantidad de escalas), `time_taken_min` (duración del vuelo), `month` y `day_of_week`.  
3. Los modelos lineales simples (Lasso) no son adecuados para esta distribución de precios.  
4. **Próximos pasos**:  
   - Tratar outliers y colas largas (Business).  
   - Evaluar transformaciones logarítmicas de `price`.  
   - Ajustar hiperparámetros de XGBoost.  
   - Aplicar la misma metodología a la clase **Economy** para comparar resultados.
